In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('/kaggle/input/playground-series-s3e7/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e7/test.csv')
submission = pd.read_csv('/kaggle/input/playground-series-s3e7/sample_submission.csv')
addition_data = pd.read_csv('/kaggle/input/reservation-cancellation-prediction/train__dataset.csv')
demand_bookings_df = pd.read_csv('/kaggle/input/hotel-booking-demand/hotel_bookings.csv')
demand_bookings_df.rename(columns = {
    'adults': 'no_of_adults',
    'children': 'no_of_children',
    'stays_in_weekend_nights': 'no_of_weekend_nights',
    'stays_in_week_nights': 'no_of_week_nights',
    'meal': 'type_of_meal_plan',
    'required_car_parking_spaces': 'required_car_parking_space',
    'reserved_room_type': 'room_type_reserved',
    'lead_time': 'lead_time',
    'arrival_date_year': 'arrival_year',
    'arrival_date_month': 'arrival_month',
    'arrival_date_day_of_month': 'arrival_date',
    'market_segment': 'market_segment_type',
    'is_repeated_guest': 'repeated_guest',
    'previous_cancellations': 'no_of_previous_cancellations',
    'previous_bookings_not_canceled': 'no_of_previous_bookings_not_canceled',
    'adr': 'avg_price_per_room',
    'total_of_special_requests': 'no_of_special_requests',
    'is_canceled': 'booking_status'
}, inplace=True)

demand_bookings_df['arrival_month'] = demand_bookings_df['arrival_month'].map({
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
})

demand_bookings_df['type_of_meal_plan'] = \
    demand_bookings_df['type_of_meal_plan'].map({
        'BB': 0, 'HB': 2, 'SC': 1, 'Undefined': 1, 'FB': 3
    })

demand_bookings_df['market_segment_type'] = \
    demand_bookings_df['market_segment_type'].map({
        "'Online TA": 1, "Offline TA/TO": 0, "Corporate": 2, "Complementary": 4, "Aviation": 3
    })

demand_bookings_df['room_type_reserved'] = \
    demand_bookings_df['room_type_reserved'].map({
        'A':  0, 'D':  1, 'E':  3, 'F':  2, 'G':  4, 'B':  5, 'C':  6
    })

demand_bookings_df['market_segment_type'].fillna(5, inplace=True)
demand_bookings_df['room_type_reserved'].fillna(7, inplace=True)


train_df['is_generated'] = 1
test_df['is_generated'] = 1
demand_bookings_df['is_generated']=0
addition_data['is_generated'] = 0


In [2]:
train_df.info()
addition_data.isna().any()
train_df = pd.concat([train_df, addition_data,demand_bookings_df],axis=0, ignore_index=True)
df = pd.concat([train_df, test_df], axis=0)
df = df.drop('id', axis=1)
df
from pandas.tseries.holiday import USFederalHolidayCalendar

def fe(df):
        # Fix date anomalies (pd.to_datetime throws parsing error for some days, see anomalies section).
        df['arrival_year_month'] = pd.to_datetime(df['arrival_year'].astype(str) + df['arrival_month'].astype(str), format='%Y%m')
        df.loc[df.arrival_date > df.arrival_year_month.dt.days_in_month, 'arrival_date'] = df.loc[df.arrival_date > df.arrival_year_month.dt.days_in_month, 'arrival_year_month'].dt.days_in_month
        df.drop(columns='arrival_year_month', inplace=True)
        
        # Creates date features.
        df['arrival_full_date'] = (df['arrival_year'].astype(str) 
                                   + '-' + df['arrival_month'].astype(str)
                                   + '-' + df['arrival_date'].astype(str))
        df['no_of_adults_div_price'] = df.no_of_adults / (df.avg_price_per_room + 1e-6)
        df['no_of_children_div_price'] = df.no_of_children / (df.avg_price_per_room + 1e-6)
        df['lead_time_div_price'] = df.lead_time / (df.avg_price_per_room + 1e-6)
        df.drop(columns=['arrival_full_date'], inplace=True)
        return df
df = fe(df)
df.isnull().any()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42100 entries, 0 to 42099
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    42100 non-null  int64  
 1   no_of_adults                          42100 non-null  int64  
 2   no_of_children                        42100 non-null  int64  
 3   no_of_weekend_nights                  42100 non-null  int64  
 4   no_of_week_nights                     42100 non-null  int64  
 5   type_of_meal_plan                     42100 non-null  int64  
 6   required_car_parking_space            42100 non-null  int64  
 7   room_type_reserved                    42100 non-null  int64  
 8   lead_time                             42100 non-null  int64  
 9   arrival_year                          42100 non-null  int64  
 10  arrival_month                         42100 non-null  int64  
 11  arrival_date   

no_of_adults                            False
no_of_children                           True
no_of_weekend_nights                    False
no_of_week_nights                       False
type_of_meal_plan                       False
required_car_parking_space              False
room_type_reserved                      False
lead_time                               False
arrival_year                            False
arrival_month                           False
arrival_date                            False
market_segment_type                     False
repeated_guest                          False
no_of_previous_cancellations            False
no_of_previous_bookings_not_canceled    False
avg_price_per_room                      False
no_of_special_requests                  False
booking_status                           True
is_generated                            False
hotel                                    True
arrival_date_week_number                 True
babies                            

In [3]:
categorical = [
    'no_of_adults',
    'no_of_children',
    'no_of_weekend_nights',
    'type_of_meal_plan',
    'required_car_parking_space',
    'room_type_reserved',
    'arrival_year',
    'arrival_month',
    'market_segment_type',
    'repeated_guest',
    'no_of_special_requests',
    "no_of_previous_cancellations",
    "is_generated"
]

continues = ['avg_price_per_room',"no_of_adults_div_price",
            "no_of_children_div_price","lead_time_div_price"]

other = [
    'no_of_week_nights',
    'lead_time',
    'arrival_date',
    #'no_of_previous_cancellations',
    'no_of_previous_bookings_not_canceled',
]

newCol=["booking_status","no_of_children","hotel"  ,
"arrival_date_week_number"                 ,
"babies"                                   ,
"country"                                  ,
"distribution_channel"                     ,
"assigned_room_type"                       ,
"booking_changes"                          ,
"deposit_type","agent"                     ,
"company"                                  ,
"days_in_waiting_list"                     ,
"customer_type"                            ,
"reservation_status"                       ,
"reservation_status_date"                  ,
"no_of_adults_div_price"        ,
 "lead_time_div_price"                      ]     
clean_list=[]
for i in newCol:
    if i in continues+other+categorical:
        print(i)
        clean_list.append(i)
df=df[df[clean_list].notna().iloc[:,0]]

no_of_children
no_of_adults_div_price
lead_time_div_price


In [4]:
df=df[continues+other+categorical+["booking_status"]].reset_index(drop=True)
df

,avg_price_per_room,no_of_adults_div_price,no_of_children_div_price,lead_time_div_price,no_of_week_nights,lead_time,arrival_date,no_of_previous_bookings_not_canceled,no_of_adults,no_of_children,...,required_car_parking_space,room_type_reserved,arrival_year,arrival_month,market_segment_type,repeated_guest,no_of_special_requests,no_of_previous_cancellations,is_generated,booking_status
0,67.50,0.029630,0.0,0.133333,2,9,14,0,2,0.0,...,0,0.0,2018,1,1.0,1,0,11,1,0.0
1,72.25,0.027682,0.0,1.619377,2,117,29,0,2,0.0,...,0,0.0,2018,7,0.0,0,0,0,1,0.0
2,52.00,0.038462,0.0,6.057692,1,315,2,0,2,0.0,...,0,0.0,2018,12,0.0,0,0,0,1,0.0
3,56.00,0.017857,0.0,0.571429,2,32,1,0,1,0.0,...,0,0.0,2018,12,1.0,0,0,0,1,0.0
4,100.00,0.020000,0.0,2.580000,0,258,16,0,2,0.0,...,0,0.0,2018,10,0.0,0,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207686,160.00,0.012500,0.0,1.968750,2,315,30,0,2,0.0,...,0,0.0,2018,9,1.0,0,0,0,1,NaN
207687,65.00,0.030769,0.0,1.246154,2,81,25,0,2,0.0,...,0,0.0,2018,3,0.0,0,1,0,1,NaN
207688,85.00,0.023529,0.0,0.470588,2,40,22,0,2,0.0,...,0,0.0,2018,10,0.0,0,0,0,1,NaN
207689,162.75,0.018433,0.0,0.024578,4,4,6,0,3,0.0,...,0,1.0,2018,9,1.0,0,0,0,1,NaN


In [5]:
train_df = df.iloc[:-len(test_df),:]
y = df['booking_status'].copy()
train_df['booking_status'] = y[:-len(test_df)]
test_df = df.iloc[-len(test_df):,:].reset_index(drop=True)

# oversample = train_df[train_df['Class']==1]
# undersample = train_df[train_df['Class']==0]

X = train_df.drop(['booking_status'], axis=1)
y = train_df.booking_status.astype('int')
X_test = test_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
features = X.columns.to_list()
uniques = []
for f in features:
    item = {'feature':f}
    count = len(X[f].unique())
    item['count'] = count
    if count < 10:
        item['values'] = X[f].unique().tolist()
    elif count < X.shape[0]:
        item['values'] = X[f].unique().tolist()[:10] + ['...']
    else:
        item['values'] = ['All values are unique']
    uniques.append(item)
df_uniques = pd.DataFrame(uniques)
df_uniques = df_uniques.set_index('feature')



print(len(continues+other+categorical)==len(df_uniques))
df_uniques.style.background_gradient(subset=['count'], cmap='Reds')


True


,count,values
feature,,
avg_price_per_room,8940,"[67.5, 72.25, 52.0, 56.0, 100.0, 212.06, 115.0, 87.0, 72.75, 117.3, '...']"
no_of_adults_div_price,12196,"[0.02962962919067216, 0.02768166051651681, 0.03846153772189351, 0.017857142538265314, 0.0199999998, 0.0, 0.017391304196597356, 0.02298850548289074, 0.027491408556819127, 0.025575447352297977, '...']"
no_of_children_div_price,3784,"[0.0, 0.009431292985799807, 0.009037505607602777, 0.008230452607156769, 0.012820512738330047, 0.006142883431765567, 0.025687130417581167, 0.0076628351903231025, 0.005837711582967241, 0.010695187108581887, '...']"
lead_time_div_price,54918,"[0.13333333135802472, 1.6193771402162334, 6.057692191198227, 0.57142856122449, 2.5799999742, 1.0138639959734792, 2.7826086714555767, 3.0459769764830233, 2.5979381086194073, 1.1679454290882743, '...']"
no_of_week_nights,35,"[2, 1, 0, 5, 3, 4, 7, 6, 8, 10, '...']"
lead_time,479,"[9, 117, 315, 32, 258, 215, 320, 265, 189, 137, '...']"
arrival_date,31,"[14, 29, 2, 1, 16, 6, 18, 24, 8, 22, '...']"
no_of_previous_bookings_not_canceled,73,"[0, 2, 3, 23, 47, 7, 8, 14, 4, 6, '...']"
no_of_adults,14,"[2, 1, 0, 3, 4, 40, 26, 50, 27, 55, '...']"


In [7]:
import catboost

gpu_params = {'task_type' : "GPU", 'devices' : '0:1'}
cbr_params = {'iterations': 2000, 
              'max_depth': 7,
              'learning_rate': 0.05, #0.1, # 0.05, 
              'verbose': 200,
               **gpu_params
             }
cbr_model = catboost.CatBoostRegressor(**cbr_params)




gpu_params = {'task_type' : "GPU", 'devices' : '0:1'}
cbc_params = {'iterations': 5000, #164,
              'max_depth': 5, #10,
              'learning_rate': 0.05, 
              'verbose': 200,
              'eval_metric': 'AUC',
              'loss_function': 'Logloss',
               **gpu_params
             }


cbc_model = catboost.CatBoostClassifier(**cbc_params)

from xgboost import XGBClassifier, XGBRegressor
xgbr_params = {'n_estimators': 2000, 
               'max_depth': 5,
               'learning_rate': 0.05, 
               'subsample': 0.8291850469303983,
                'tree_method': 'gpu_hist',
               'verbose': 200,
              }
xgbr_model = XGBRegressor(**xgbr_params)
import lightgbm as lgbm
gpu_params = {'device' : "gpu"}
gbc_params = {'n_estimators': 1000, 
              'max_depth': 5, #10, 
              'num_leaves': 8,
              'learning_rate': 0.05, 
              #'subsample': 0.8501198417003352,
             # 'lambda_l1': 3,
             # 'lambda_l2': 3,
             # 'bagging_fraction': 0.8,
             # 'feature_fraction': 0.8,
              'objective': 'binary',
              'metric': 'auc',
              'verbose': 0,
               **gpu_params
             }
lgbm_model = lgbm.LGBMClassifier(**gbc_params)
#lgbr_model = lgbm.LGBMRegressor(**lgbr_params)

xgbc_params = {'n_estimators': 1000, #54, 
               'max_depth': 5, #10, #3, 
               'learning_rate': 0.05, 
               #'subsample': 0.568355005569169, 
               'eval_metric'     : 'auc',
               'objective'       : 'binary:logistic',
                'tree_method': 'gpu_hist',
               #'verbose': 2000,
              }
xgbc_model = XGBClassifier(**xgbc_params)

from sklearn.linear_model import LassoCV, Lasso
LassoCV_MODEL_PARAMS = {
                       'precompute': "auto",
                       'fit_intercept': True,
                       'normalize': False,
                       'max_iter': 1000,
                    
                       'eps': 0.0001,
                       'cv': 5,
                       'n_alphas': 1000,
                       'n_jobs': 8,
                    'verbose': 200,
#                        'tol': 0.0001
}
LassoCV_model = LassoCV(**LassoCV_MODEL_PARAMS)

import catboost
gpu_params = {'task_type' : "GPU", 'devices' : '0:1'}
cbc_params = {'iterations': 1000, #164,
              'max_depth': 5, #10,
              'learning_rate': 0.05, 
              'verbose': 200,
              'eval_metric': 'Logloss',
              
              'loss_function': 'Logloss',
               **gpu_params
             }
cbc_model = catboost.CatBoostClassifier(**cbc_params)

from sklearn.ensemble import RandomForestClassifier
Forest_model = RandomForestClassifier(min_samples_split=3, 
                             #max_leaf_nodes=106,
                             criterion = 'gini',
                             n_estimators=500,
                             max_depth=15,
                             max_features=10
                            )


In [8]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
sub=True

if sub:
    X_train = train_df.drop(['booking_status'], axis=1)
    y_train = train_df.booking_status.astype('int')
    X_test = test_df
else : X_train, X_test, y_train, y_test = train_test_split(X ,y, random_state=42, test_size=.2)
    
# Define the column transformer for preprocessing the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continues+other),
        ('cat', OneHotEncoder(handle_unknown='ignore'),categorical)
    ])
# Define the base models
base_models = [
    ("cbc_model",cbc_model),
    ("xgbc_model",xgbc_model),
    ("lgbm_model",lgbm_model),
    ("Forest_model",Forest_model)
    #('lr', LogisticRegression()),
    #('dt', DecisionTreeClassifier())
]

# Define the meta-model
meta_model = LogisticRegression()

# Define the stacking classifier with k-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=kfold#,n_jobs=15
                                 )

# Define the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('stacking_clf', stacking_clf)])

# Train the pipeline on the full training set
pipeline.fit(X_train, np.ravel(y_train),)

# Use the pipeline to make predictions on new, unseen data
y_pred = pipeline.predict(X_test)

# Evaluate the accuracy of the predictions
if not sub:
    accuracy = accuracy_score(y_test, y_pred)
    print('Accuracy:', accuracy)

0:	learn: 0.6737816	total: 13ms	remaining: 13s
200:	learn: 0.4473989	total: 1.69s	remaining: 6.73s
400:	learn: 0.4288253	total: 3.37s	remaining: 5.04s
600:	learn: 0.4173999	total: 5.44s	remaining: 3.61s
800:	learn: 0.4092028	total: 7.12s	remaining: 1.77s
999:	learn: 0.4026835	total: 8.76s	remaining: 0us
0:	learn: 0.6740603	total: 9.52ms	remaining: 9.51s
200:	learn: 0.4478323	total: 1.72s	remaining: 6.85s
400:	learn: 0.4287381	total: 3.39s	remaining: 5.07s
600:	learn: 0.4175332	total: 5.04s	remaining: 3.35s
800:	learn: 0.4092132	total: 6.7s	remaining: 1.67s
999:	learn: 0.4027759	total: 8.36s	remaining: 0us
0:	learn: 0.6737304	total: 11.4ms	remaining: 11.4s
200:	learn: 0.4460064	total: 2.86s	remaining: 11.4s
400:	learn: 0.4276983	total: 4.54s	remaining: 6.78s
600:	learn: 0.4160839	total: 6.2s	remaining: 4.11s
800:	learn: 0.4075366	total: 7.86s	remaining: 1.95s
999:	learn: 0.4013079	total: 9.5s	remaining: 0us
0:	learn: 0.6738491	total: 9.01ms	remaining: 9s
200:	learn: 0.4468506	total: 2.1

In [9]:

submission['booking_status'] = y_pred
submission


,id,booking_status
0,42100,0
1,42101,0
2,42102,0
3,42103,0
4,42104,1
...,...,...
28063,70163,1
28064,70164,0
28065,70165,0
28066,70166,1


In [10]:
submission.to_csv('submission.csv', index=False)